In [4]:
from openai import OpenAI
import os

MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 0822

https://platform.openai.com/docs/guides/fine-tuning

서설

프롬프트 엔지니어링으로 처리해보고 안될때 파인튜닝 해볼수있다. 

더 많은 예제로 훈련 가능, 대화형으로 훈련 가능 - 대화가 많은면 퓨샷으로 처리하는게 의미가 없다.

대화가 많으니 파인튜닝으로 돌린 모델 만들어 놓기

퓨샷이 많으면 쿼리때마다 퓨샷만큼의 토큰을 계속 사용

데이터셋으로 만들어서 파인튜닝

데이터셋이 일관되지 않으면 할루시네이션 발생

대화를 하다보면 시퀀스가 중요함, 예) 인사를 한 후 물어보는게 자연스러움, 점차 난이도 증가시키면서 종료

파인튜닝은 개별 데이터이기 때문에 시퀀스를 반영하기 어려웠으나 지침을 잘 내린다면 가능, 그리고 멀티턴 챗으로 훈련하는 것도 좋음

래그를 같이 사용하는 경우 처리방법: 구체적인 정보가 필요한 경우에는 래그를 사용하고, 파인튜닝도 가능하나 효율적이지 않음, 지식기반 모델이면 지양

지식 기반은 파인튜닝으로 몇백개의 데이터 포인트로 만들어야해서 한계가 있음

제이슨 리스트 타입으로 분류

훈련 테스트 분리 하여 검증 가능





## How to fine-tune chat models

미세 조정은 프롬프트에 맞출 수 있는 것보다 훨씬 더 많은 예제를 학습하여 모델을 개선하고, 다양한 작업에서 더 나은 결과를 얻을 수 있도록 합니다. 이 노트북은 새로운 GPT-4o 미니 미세 조정에 대한 단계별 가이드를 제공합니다. 다양한 레시피와 각각에 대한 추출된 일반 재료 목록을 제공하는 RecipeNLG 데이터 세트를 사용하여 엔터티 추출을 수행합니다 . 이는 명명된 엔터티 인식(NER- 개체명 인식name entity recognization) 작업에 공통적인 데이터 세트입니다.

다음 단계를 살펴보겠습니다.

- 설정: 데이터 세트를 로드하고 미세 조정할 하나의 도메인으로 필터링합니다.
- 데이터 준비: 훈련 및 검증 사례(샘플)를 만들고 이를 Files엔드포인트에 업로드하여 미세 조정을 위한 데이터를 준비합니다.
- 미세 조정: 미세 조정된 모델을 만듭니다.
- 추론: 미세 조정된 모델을 사용하여 새로운 입력에 대한 추론을 수행합니다.
이 과정을 마치면 미세 조정된 gpt-4o-mini-2024-07-18모델을 훈련, 평가하고 배포할 수 있게 됩니다.

미세 조정에 대한 자세한 내용은 설명서 가이드 나 API 참조를 참조하세요 .

chat completion api 기반 프롬프트로 모델 생성

In [5]:
# zero-shot completions : llm이 예시없이 바로 문장 생성

from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.",
        },
        {
            "role": "user",
            "content": 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: ',
        },
    ],
)

print(response.choices[0].message.content)

- Brown sugar
- Evaporated milk
- Vanilla
- Nuts (pecans)
- Butter or margarine
- Rice biscuits


In [10]:
# few-shot completions

from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.",
        },
        {
            "role": "user",
            "content": 'Title: Classic Chocolate Chip Cookies\n\nIngredients: ["1 c. unsalted butter", "3/4 c. granulated sugar", "3/4 c. packed brown sugar", "1 tsp. vanilla extract", "2 large eggs", "2 1/4 c. all-purpose flour", "1/2 tsp. baking soda", "1/2 tsp. salt", "2 c. chocolate chips"]\n\nGeneric ingredients: ',
        },
        {
            "role": "assistant",
            "content": """["brown sugar", "evaporated milk", "vanilla", "nuts", "butter", "rice biscuits"  ]""",
        },
        
        
    ],
)

print(response.choices[0].message.content)

["butter", "sugar", "vanilla", "eggs", "flour", "baking soda", "salt", "chocolate chips"]


퓨샷으로 많이 해보고 파인튜닝 데이터셋을 만들면 됨

파인 튜닝(fine-tuning)은 특정 도메인에 집중할 때 가장 효과적입니다. 모델이 학습할 수 있도록 데이터셋이 충분히 집중되어 있어야 하지만, 새로운 예시를 놓치지 않도록 어느 정도 일반성도 유지하는 것이 중요합니다. 이를 염두에 두고, 우리는 RecipesNLG 데이터셋에서 www.cookbooks.com의 문서만 포함하는 하위 집합을 추출했습니다.

In [12]:
import json
import os
from pprint import pprint
import pandas as pd
recipe_df = pd.read_csv(r"D:\pythonProject\ML\data\cookbook_recipes_nlg_10k.csv")
recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [13]:
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        10000 non-null  object
 1   ingredients  10000 non-null  object
 2   directions   10000 non-null  object
 3   link         10000 non-null  object
 4   source       10000 non-null  object
 5   NER          10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


## Data preparation

우리는 데이터를 준비하는 것부터 시작할 것입니다. ChatCompletion 형식으로 파인 튜닝할 때, 각 학습 예시는 메시지들의 단순한 목록으로 구성됩니다. 예를 들어, 하나의 항목은 다음과 같이 생겼을 수 있습니다:

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```


훈련 과정에서 이 대화는 나뉘게 되며, 마지막 항목은 모델이 생성할 답변(완성)이 되고, 나머지 메시지들은 프롬프트 역할을 합니다. 따라서 학습 예제를 만들 때 이를 고려해야 합니다. 모델이 여러 차례 주고받는 대화에서 작동할 경우, 대화가 확장되더라도 성능이 떨어지지 않도록 대표적인 예시들을 제공해야 합니다.


In [14]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def create_user_message(row):
    return f"""Title: {row["title"]}\n\nIngredients: {row["ingredients"]}\n\nGeneric ingredients: """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})
    
    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})
    
    messages.append({"role": "assistant", "content": row["NER"]})
    
    return {"messages": messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


로데이터를 가공해서 jsonl 파일로 저장해야함

In [16]:
training_df = recipe_df.loc[0:20]

training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["bee

훈련 데이터 외에 선택적으로 검증 데이터를 제공할 수도 있습니다. 검증 데이터는 모델이 훈련 세트에 과도하게 적합하지 않은지 확인하는 데 사용됩니다.

In [18]:
validation_df = recipe_df.loc[20:30]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

jsonl은 파일의 각줄에 하나의 json 객체를 가지는 형식입니다. 이 형식은 대규모 데이터셋을 처리할 때 효율적입니다. 이 예제에서는 훈련 및 검증 데이터를 준비하기 위해 jsonl 파일을 사용합니다.

In [19]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"  #한줄씩 저장
            out.write(jout)

In [20]:
training_file_name = "tmp_recipe_finetune_trinng.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [21]:
!head -n 5 tmp_recipe_finetune_trinng.jsonl

'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## Fine-tuning

In [25]:
from openai import OpenAI

client = OpenAI()

train = client.files.create(
    file=open(r"D:\pythonProject\ML\tmp_recipe_finetune_trinng.jsonl", "rb"),
    purpose="fine_tune",
)

valid = client.files.create(
    file=open(r"D:\pythonProject\ML\tmp_recipe_finetune_validation.jsonl", "rb"),
    purpose="fine_tune",
)

BadRequestError: Error code: 400 - {'error': {'message': "'fine_tune' is not one of ['fine-tune', 'assistants', 'batch', 'user_data', 'responses', 'vision'] - 'purpose'", 'type': 'invalid_request_error', 'param': 'purpose', 'code': None}}

In [27]:
from openai import OpenAI

MODEL = "gpt-4o-mini-2024-07-18"
client = OpenAI(
)
train = client.files.create(
    file=open(r"D:\pythonProject\ML\tmp_recipe_finetune_trinng.jsonl", "rb"),
    purpose="fine-tune",
)
valid = client.files.create(
    file=open(r"D:\pythonProject\ML\tmp_recipe_finetune_validation.jsonl", "rb"),
    purpose="fine-tune",
)

In [28]:
train

FileObject(id='file-Z2YnJPzpMOmNjq3EI2dK4Z0D', bytes=12374, created_at=1724293484, filename='tmp_recipe_finetune_trinng.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [29]:
valid

FileObject(id='file-ylcFuWicjw9VBqL3Awsk75r9', bytes=6377, created_at=1724293485, filename='tmp_recipe_finetune_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [33]:
client.fine_tuning.jobs.create(
    model=MODEL,
    training_file=train.id,
    validation_file=valid.id,
    hyperparameters={"n_epochs": 3},
)

FineTuningJob(id='ftjob-55aGg2nfwcugmuQt9Qo75KA5', created_at=1724293663, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-NXueW4KXYYo6IkVSQ8lGCen3', result_files=[], seed=1162411322, status='validating_files', trained_tokens=None, training_file='file-Z2YnJPzpMOmNjq3EI2dK4Z0D', validation_file='file-ylcFuWicjw9VBqL3Awsk75r9', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [47]:
response = client.fine_tuning.jobs.retrieve('ftjob-55aGg2nfwcugmuQt9Qo75KA5')
print(response)

FineTuningJob(id='ftjob-55aGg2nfwcugmuQt9Qo75KA5', created_at=1724293663, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::9yrwFQRd', finished_at=1724294045, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-NXueW4KXYYo6IkVSQ8lGCen3', result_files=['file-GCsZPeN4uxCFtyzvxFseDrgR'], seed=1162411322, status='succeeded', trained_tokens=8313, training_file='file-Z2YnJPzpMOmNjq3EI2dK4Z0D', validation_file='file-ylcFuWicjw9VBqL3Awsk75r9', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [48]:
status = response.status

if status == "succeeded":
    print("Fine-tuning job completed successfully")

elif status == "failed":
    print("Fine-tuning job failed")

else:
    print(f"Fine-tuning job is {status}")

total_tokens = response.trained_tokens
print(f"Total tokens processed: {total_tokens}")

Fine-tuning job completed successfully
Total tokens processed: 8313


훈련 데이터 외에 선택적으로 검증 데이터를 제공할 수도 있습니다. 검증 데이터는 모델이 훈련 세트에 과도하게 적합하지 않은지 확인하는 데 사용됩니다.

마지막 단계는 추론을 위해 미세 조정된 모델을 사용하는 것입니다. FineTuning 클래식과 유사하게 모델 매개변수를 채우는 새로운 미세 조정된 모델 이름으로 `ChatCompletions`을 호출하기만 하면 됩니다

In [49]:
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-55aGg2nfwcugmuQt9Qo75KA5', created_at=1724293663, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::9yrwFQRd', finished_at=1724294045, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-NXueW4KXYYo6IkVSQ8lGCen3', result_files=['file-GCsZPeN4uxCFtyzvxFseDrgR'], seed=1162411322, status='succeeded', trained_tokens=8313, training_file='file-Z2YnJPzpMOmNjq3EI2dK4Z0D', validation_file='file-ylcFuWicjw9VBqL3Awsk75r9', estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [50]:
test_df = recipe_df.loc[31:40]
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": create_user_message(test_row)})
pprint(test_messages)    

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Summer Chicken\n'
             '\n'
             'Ingredients: ["1 pkg. chicken cutlets", "1/2 c. oil", "1/3 c. '
             'red vinegar", "2 Tbsp. oregano", "2 Tbsp. garlic salt"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


### Full Validation Loss의 목적:
- 과적합 방지: 훈련 데이터에 대한 손실(training loss)은 낮을 수 있지만, 검증 데이터에 대한 성능이 떨어지면 모델이 과적합되고 있는 신호입니다. Full validation loss는 이런 과적합 여부를 감지하는 데 유용합니다.
- 모델 선택: 파인튜닝 중 다양한 하이퍼파라미터 조합을 실험할 때, Full validation loss가 가장 낮은 지점에서 최적의 모델을 선택하는 기준이 될 수 있습니다.

Full validation loss는 파인튜닝 중 모델이 검증 데이터셋에 대해 얼마나 잘 예측하는지를 평가하는 중요한 지표로, 모델의 성능과 일반화 능력을 판단하는 데 도움을 줍니다.

## inference

마지막 단계는 추론을 위해 미세 조정된 모델을 사용하는 것입니다. FineTuning 클래식과 유사하게 모델 매개변수를 채우는 새로운 미세 조정된 모델 이름으로 `ChatCompletions`을 호출하기만 하면 됩니다

정확한 평가를 위해서는 별도의 평가 데이터와 툴을 만들어서 확인해야함

모범 답안과 예측과의 유사도를 측정하여 성능을 평가할 수 있습니다. 이를 위해 두 문장 간의 유사도를 측정하는 함수를 작성하고, 이 함수를 사용하여 예측과 모범 답안 간의 유사도를 계산합니다. 이를 통해 모델의 성능을 정량적으로 평가할 수 있습니다.

애플리케이션 개발할때 평가를 어떻게 할지 고민을 해봐야 함 - 실제 성능

이상적인 답안 만들기 - 레이블링

그 임계치가 일정 수준 이상이면 성능이 좋다고 판단할 수 있음

In [51]:
completion = client.chat.completions.create(
    model='ft:gpt-4o-mini-2024-07-18:personal::9yrwFQRd',
    messages = test_messages,
    temperature=0,
    max_tokens=500
)

print(completion.choices[0].message.content)

["chicken", "oil", "red vinegar", "oregano", "garlic salt"]
